# 1) Chargement des données

Le jeu de données GOAT_all_fin est issu d'un ensemble de transformation que l'on a réalisé sur le jeu d'origine se trouvant sur le site kaggle:
https://www.kaggle.com/hugomathien/soccer

In [1]:
import pandas as pd
mon_enregistrement=pd.read_csv('GOAT_all_fin.csv')
mon_enregistrement.tail()
mon_enregistrement.shape

(13801, 52)

In [2]:
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
import warnings

In [3]:
try:
    from sklearn.preprocessing import OrdinalEncoder
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder
from sklearn import preprocessing    

# 2) Création d'un jeu d'entrainement et de test

In [4]:
matches=mon_enregistrement
X_df=matches.drop(['label'],axis=1)
Y_df=matches['label']
X_df.shape

(13801, 51)

In [5]:
# on fait un one-hot-encoder pour la variable correspondant au championnat.
matches_cat=X_df['league_name']
enc = preprocessing.LabelEncoder()
enc.fit(matches_cat)
new_cat_features = enc.transform(matches_cat)
print (new_cat_features) # [1 2 0]
new_cat_features = new_cat_features.reshape(-1, 1) # On redimensionne
ohe = preprocessing.OneHotEncoder(sparse=False) #Plus facile à lire
data_2=ohe.fit_transform(new_cat_features)
data_2.shape

[1 1 1 ... 4 4 0]


C:\Users\LeLimier\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(13801, 5)

In [6]:
#on convertit le tableau numpy en DataFrame.
data_2=pd.DataFrame(data_2)
data_2.shape

(13801, 5)

In [7]:
#on enlève toutes les colonnes qui ne sont pas utiles pour l'entrainement et la prédiction.
X_df=X_df.drop(['home_team','away_team','stage','country_name','league_name','season','date','id','home_team_goal','away_team_goal'],axis=1)

In [8]:
X_df=pd.concat([data_2,X_df],axis=1)
X_df.shape

(13801, 46)

In [9]:
#On constitue un jeu de test et d'entrainement.
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, test_size = 0.2, random_state = 42,stratify = Y_df)

In [10]:
col_paris=['B365H',
'B365D',
'B365A',
'BWH',
'BWD',
'BWA',
'IWH',
'IWD',
'IWA',
'LBH',
'LBD',
'LBA',
'PSH',
'PSD',
'PSA']

In [11]:
#On constitue le DataFrame des paris qu'on utilisera une fois avoir trouvé notre modèle.
les_paris=X_test[col_paris]
X_train=X_train.drop(col_paris,axis=1)#on enlève les cotes des paris pour l'entrainement.
X_test=X_test.drop(col_paris,axis=1)

In [12]:
X_train.head()

,0,1,2,3,4,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,...,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,vict_cons_dom,inv_dom,vict_cons_ext,inv_ext
1766,0.0,1.0,0.0,0.0,0.0,75.0,65.0,71.0,70.0,71.0,...,76.0,58.0,75.0,75.0,73.0,74.0,1,1,-1,1
2251,1.0,0.0,0.0,0.0,0.0,77.0,75.0,77.0,78.0,75.0,...,73.0,70.0,72.0,74.0,70.0,74.0,4,5,-1,-1
11936,0.0,0.0,1.0,0.0,0.0,80.0,72.0,74.0,77.0,74.0,...,79.0,77.0,76.0,81.0,76.0,78.0,-2,9,1,1
9319,0.0,0.0,0.0,0.0,1.0,81.0,78.0,71.0,80.0,72.0,...,70.0,70.0,67.0,80.0,80.0,75.0,3,5,-7,-5
781,0.0,0.0,0.0,0.0,1.0,81.0,68.0,74.0,71.0,67.0,...,73.0,77.0,66.0,76.0,72.0,78.0,-2,1,-2,-2


# 3) Modélisation

In [13]:
Rand_clf = RandomForestClassifier(n_estimators = 100, random_state = 1, class_weight = 'balanced')
Ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 2)
KNN_clf =  KNeighborsClassifier()
GNB_clf = GaussianNB()


clfs = [Rand_clf, Ada_clf,  KNN_clf, GNB_clf]

#On établit une grille de paramètres pour nos classifieurs afin de trouver le modèle le plus
# optimal avec une recherche en grille.
feature_len = X_train.shape[1]
scorer = make_scorer(accuracy_score)
params_Rand = {'clf__max_features': ['auto', 'log2'], 
                 'dm_reduce__n_components': np.arange(8, feature_len, np.around(feature_len/4))}
params_Ada = {'clf__learning_rate': np.linspace(0.5, 2, 5), 
                 'dm_reduce__n_components': np.arange(8, feature_len, np.around(feature_len/4))}
params_KNN = {'clf__n_neighbors': [3, 5, 10], 
                  'dm_reduce__n_components': np.arange(8, feature_len, np.around(feature_len/4))}
params_GNB = {'dm_reduce__n_components': np.arange(8, feature_len, np.around(feature_len/4))}


params = {clfs[0]: params_Rand,
              clfs[1]: params_Ada,
              clfs[2]: params_KNN,
              clfs[3]: params_GNB,
              }

#On initialise notre pca qui sera notre paramètre de réduction de dimensionnalité.
pca = PCA()
dm_reductions = [pca]  

In [14]:
#on utilise tous les coeurs de notre ordinateur.
n_jobs = -1
cv=5 #nombre de bloc pour la validation.

# 4) Evaluation

In [15]:
def train_classifier(clf, dm_reduction, X_train, y_train, cv, params, scorer, jobs, 
                     best_components = None, best_params = None):
    
    
    #on chronomètre
    start = time()
    
    #On définit un pipeline de transformation en explicitant le classifieurs et le type de réduction de dimensions.
    ele = [('dm_reduce', dm_reduction), ('clf', clf)]
    pipeline = Pipeline(ele)
        
    params['dm_reduce__n_components'] = params['dm_reduce__n_components'].astype(int)
    #On trouve le meilleur classifieur avec une recherche en grille.
    grid_obj = model_selection.GridSearchCV(pipeline, param_grid = params, scoring = scorer, cv = cv, n_jobs = jobs)
    grid_obj.fit(X_train, y_train)
    best_pipe = grid_obj.best_estimator_
    
        
    end = time()
    
    #affiche les résultats
    print("Entrainé {} en {:.1f} minutes".format(clf.__class__.__name__, (end - start)/60))
    
    #On retourne le meilleurs classifieurs par type de modèle.
    return best_pipe
    


In [16]:
def train_predict(clf, dm_reduction, X_train, y_train, X_test, y_test, cv, params, scorer, jobs, **kwargs):
    
    
    #on donne le type de classifieur et la méthode de réduction de dimensions.
    print("On entraine un {} avec {}...".format(clf.__class__.__name__, dm_reduction.__class__.__name__))
    
    #On entraine le classifieur
    best_pipe = train_classifier(clf, dm_reduction, X_train, y_train, cv, params, scorer, jobs)
    
    clf.fit(best_pipe.named_steps['dm_reduce'].transform(X_train), y_train)
    
    
    # On affiche les résultats
    print("Score du {} pour le jeu d'entrainement: {:.4f}.".format(clf.__class__.__name__, predict_labels(clf, best_pipe, X_train, y_train)))
    print("Score du {} pour le jeu de test: {:.4f}.".format(clf.__class__.__name__, predict_labels(clf, best_pipe, X_test, y_test)))
    
    #Return classifier, dm reduction, and label predictions for train and test set
    return clf, best_pipe.named_steps['dm_reduce'], predict_labels(clf, best_pipe, X_train, y_train), predict_labels(clf, best_pipe, X_test, y_test)
        

In [17]:
def predict_labels(clf, best_pipe, X, y):
    y_pred = clf.predict(best_pipe.named_steps['dm_reduce'].transform(X))
    #Retourne le score de précision
    return accuracy_score(y.values, y_pred)

In [18]:
def find_best_classifier(classifiers, dm_reductions, scorer, X_t, y_t, X_v, y_v, cv, params, jobs):
    '''on fait le tour de tout les classifieurs et les types de réduction de dimensions '''
    
    #on initialise les tableaux dans lesquels on met nos résultats 
    clfs_return = []
    dm_reduce_return = []
    train_scores = []
    test_scores = []
    
    #on boucle sur les types de réduction de dimensions
    for dm in dm_reductions:
        
        #on boucle sur les types de classifieurs
        for clf in clfs:
            
            #On fait la recherche en grille
            clf, dm_reduce, train_score, test_score = train_predict(clf = clf, dm_reduction = dm, X_train = X_t, y_train = y_t,
                                                      X_test = X_v, y_test = y_v, cv = cv,
                                                      params = params[clf], scorer = scorer, jobs = jobs)
            
            #on complète les tableaux des résultats            
            clfs_return.append(clf)
            dm_reduce_return.append(dm_reduce)
            train_scores.append(train_score)
            test_scores.append(test_score)
    
    #On retourne les résultats
    return clfs_return, dm_reduce_return, train_scores, test_scores

In [19]:

#On entraine tous les classifieurs et on les compare.
clfs, dm_reductions, train_scores, test_scores = find_best_classifier(clfs, dm_reductions, scorer, X_train, y_train, 
                                                                     X_test, y_test, cv, 
                                                                      params, n_jobs)


On entraine un RandomForestClassifier avec PCA...
Entrainé RandomForestClassifier en 1.1 minutes
Score du RandomForestClassifier pour le jeu d'entrainement: 1.0000.
Score du RandomForestClassifier pour le jeu de test: 0.5187.
On entraine un AdaBoostClassifier avec PCA...
Entrainé AdaBoostClassifier en 1.9 minutes
Score du AdaBoostClassifier pour le jeu d'entrainement: 0.5361.
Score du AdaBoostClassifier pour le jeu de test: 0.5270.
On entraine un KNeighborsClassifier avec PCA...
Entrainé KNeighborsClassifier en 0.3 minutes
Score du KNeighborsClassifier pour le jeu d'entrainement: 0.6254.
Score du KNeighborsClassifier pour le jeu de test: 0.4440.
On entraine un GaussianNB avec PCA...
Entrainé GaussianNB en 0.0 minutes
Score du GaussianNB pour le jeu d'entrainement: 0.5257.
Score du GaussianNB pour le jeu de test: 0.5255.


In [20]:
clfs, dm_reductions, train_scores, test_scores

([RandomForestClassifier(bootstrap=True, class_weight='balanced',
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, min_impurity_decrease=0.0,
                         min_impurity_split=None, min_samples_leaf=1,
                         min_samples_split=2, min_weight_fraction_leaf=0.0,
                         n_estimators=100, n_jobs=None, oob_score=False,
                         random_state=1, verbose=0, warm_start=False),
  AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                     n_estimators=100, random_state=2),
  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                       metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                       weights='uniform'),
  GaussianNB(priors=None, var_smoothing=1e-09)],
 [PCA(copy=True, iterated_power='auto', n_components=24, random_state=None,
      svd_solver='auto', tol=0.0, 

In [21]:
y_pred=clfs[np.argmax(test_scores)].predict_proba(dm_reductions[np.argmax(test_scores)].transform(X_test))
# on calcule la probabilité prédite pour chaque classe en utilisant notre meilleur classifieur (Adaboost) avec la meilleure methode
# de réduction de dimensions(ACP à 8 axes).

In [22]:
y_pred.shape #on vérifie les dimensions (il y a 3 colonnes correspondant aux probabilités pour les 3 classes defeat, draw, win dans cet ordre)

(2761, 3)

# 5) Calcul des gains en fonction des cotes

In [23]:
y_test_np=y_test.values #on convertit le Dataframe en tableau numpy
y_test_np.shape , les_paris.shape # on vérifie les tailles

((2761,), (2761, 15))

In [24]:
les_paris.fillna(0) #on remplace les cotes qu'on ne connait pas par 0.(valeur manquante)
les_paris=les_paris.values #on convertit en tableau numpy.

In [25]:
sum=0
nb1=0
nb2=0
for j in range (len(y_test_np)):

    for k in range (5):        #on boucle sur le nombre d'organismes de paris=5
        for i in range(3):

            h= ((les_paris[j][i+3*k]-1)*y_pred[j][2-i])-(1-y_pred[j][2-i]) # on calcul l'éspérance de gain h si elle est supérieure à 2 , on simule un pari fictif
            if (h>2) and ((y_test_np[j]=='win' and i==0) or (y_test_np[j]=='draw' and i==1) or (y_test_np[j]=='defeat' and i==2))  :
                sum=sum+(les_paris[j][i+3*k]-1)
                nb1=nb1+1
            elif (h>2) and ((y_test_np[j]=='win' and i!=0) or (y_test_np[j]=='draw' and i!=1) or (y_test_np[j]=='defeat' and i!=2)):    
                sum=sum-1
                nb2=nb2+1
print('notre algorithme fait '+str(sum)+' euros avec tout les organismes en pariant 1euro sur chaque paris à espérance de gain positive \navec un nombre de paris gagnants nb1='+str(nb1)+' et un nombre de paris perdants nb2='+str(nb2))           
            

notre algorithme fait -429.59000000000003 euros avec tout les organismes en pariant 1euro sur chaque paris à espérance de gain positive 
avec un nombre de paris gagnants nb1=58 et un nombre de paris perdants nb2=1075


# Conclusion:

Notre prédicteur ne nous permet pas d'engendrer des gains positifs, on en déduit que le football est un sport trop incertain et les cotes sont trop bien ajustées par les bookmakers pour que l'on puisse tomber sur des paris réellement intéressants.
On peut aussi considérer que nos données ne synthétisent pas suffisamment l'information qui caractérise le futur résultat d'un match , il nous faudrait des données tenant compte de la mise en place tactique des deux équipes ainsi que leur historique sur une période beaucoup plus longue , on a aussi un manque de données global car il y a peu de confrontations entre deux même équipes dans une saison et donc pas la possibilité de repérer des pattern dans la succession des résultats entre deux même équipes.
De plus les algorithmes de machine Learning s'avèrent assez peu complexes mais je doute que l'on obtienne de meilleurs résultats avec des réseaux de neurones.